In [2]:
import os, cv2, pytesseract, boto3, spacy, pandas as pd, csv, json
from pdf2image import convert_from_path
from PIL import Image
from pymongo import MongoClient
from bson.binary import Binary
from IPython.display import display, Markdown

# === CONFIGURATION ===
pdf_folder = "input_pdfs"
output_img_root = "output_images"
output_csv_root = "output_csv"
output_fig_root = "output_images"
poppler_path = r"C:\Users\harik\projects\OCR_Project\poppler\Library\bin"
tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract.exe"
mongo_uri = "mongodb://localhost:27017/"
db_name = "pdf_data"

pytesseract.pytesseract.tesseract_cmd = tesseract_cmd
textract = boto3.client('textract')
nlp = spacy.load("en_core_web_sm")
client = MongoClient(mongo_uri)
db = client[db_name]

# === HELPERS (unchanged) ===
def analyze_width_for_para(block1, block2):
    b1, b2 = block1['Geometry']['BoundingBox'], block2['Geometry']['BoundingBox']
    s, l = (b1, b2) if b1['Width'] < b2['Width'] else (b2, b1)
    s_left, s_right = s['Left'], s['Left'] + s['Width']
    l_left, l_right = l['Left'], l['Left'] + l['Width']
    s_width = s['Width']
    if s_right <= l_left or s_left >= l_right:
        return 0
    elif s_left >= l_left and s_right <= l_right:
        return 100
    elif s_left <= l_left and s_right <= l_right:
        return (s_right - l_left) / s_width
    elif s_left >= l_left and s_right >= l_right:
        return (l_right - s_left) / s_width
    return 0

def analyze_height_for_para(upper, lower):
    u, l = upper['Geometry']['BoundingBox'], lower['Geometry']['BoundingBox']
    return (l['Top'] - (u['Top'] + u['Height'])) / u['Height'] if u['Height'] != 0 else 0

def is_basic_svo(sentence): return any(t.pos_ == "VERB" for t in nlp(sentence))
def get_text_from_Id(lines, id): return next((b['Text'] for b in lines if b['Id'] == id), "")
def get_block_from_Id(lines, id): return next((b for b in lines if b['Id'] == id), None)
def correct_coordinates(lines, w, h):
    for b in lines:
        bb = b['Geometry']['BoundingBox']
        bb.update({k: int(bb[k] * w if 'Width' in k or 'Left' in k else bb[k] * h) for k in bb})

def is_long_sentence(text): return len(text.split()) > 4
def calculate_average_font_size(lines): return sum(b['Geometry']['BoundingBox']['Height'] for b in lines) / len(lines)
def list_headings(lines, pct):
    avg = calculate_average_font_size(lines)
    return [b['Id'] for b in lines if b['Geometry']['BoundingBox']['Height'] > avg * (1 + pct / 100) and b['Text'][0].isupper()]

# === TABLE LOGIC ===
def get_table_data(table, blocks_map):
    rows = {}
    for rel in table.get('Relationships', []):
        if rel['Type'] == 'CHILD':
            for cid in rel['Ids']:
                cell = blocks_map[cid]
                if cell['BlockType'] == 'CELL':
                    row, col = cell['RowIndex'], cell['ColumnIndex']
                    rows.setdefault(row, {})[col] = get_cell_text(cell, blocks_map)
    sorted_rows = sorted(rows.items())
    max_cols = max(len(r[1]) for r in sorted_rows)
    return [[cols.get(c, '') for c in range(1, max_cols + 1)] for _, cols in sorted_rows]

def get_cell_text(cell, blocks_map):
    txt = ''
    for rel in cell.get('Relationships', []):
        for cid in rel.get('Ids', []):
            b = blocks_map[cid]
            if b['BlockType'] == 'WORD': txt += b['Text'] + ' '
            elif b['BlockType'] == 'SELECTION_ELEMENT' and b['SelectionStatus'] == 'SELECTED': txt += 'X '
    return txt.strip()

# === CONVERT & ROTATE ===
def convert_pdf_to_images(pdf_path, output_folder):
    pdf_name = os.path.splitext(os.path.basename(pdf_path))[0]
    os.makedirs(output_folder, exist_ok=True)
    images = convert_from_path(pdf_path, poppler_path=poppler_path)
    return [os.path.join(output_folder, f"{pdf_name}_page_{i+1}.png") for i, img in enumerate(images) if not img.save(os.path.join(output_folder, f"{pdf_name}_page_{i+1}.png"), 'PNG')]

def correct_image_rotation(path):
    img = Image.open(path)
    try:
        angle = pytesseract.image_to_osd(img, output_type=pytesseract.Output.DICT).get('rotate', 0)
        if angle not in [0, 360]:
            img.rotate(-angle, expand=True).save(path)
            print(f"🔄 Rotated {angle}°: {path}")
    except Exception as e:
        print(f"⚠️ Rotation detection failed: {e}")
    return path

# === EXTRACT TABLES ===
def extract_tables(image_path, document_id, page_number):
    with open(image_path, "rb") as doc_file:
        image_bytes = doc_file.read()
    response = textract.analyze_document(Document={'Bytes': image_bytes}, FeatureTypes=["TABLES"])
    blocks = response['Blocks']
    blocks_map = {b['Id']: b for b in blocks}
    tables = [b for b in blocks if b['BlockType'] == 'TABLE']
    structured_tables, raw_tables = [], []
    image = cv2.imread(image_path)
    h, w = image.shape[:2]
    csv_dir = os.path.join(output_csv_root, document_id, "tables")
    os.makedirs(csv_dir, exist_ok=True)

    for i, table in enumerate(tables):
        rows = get_table_data(table, blocks_map)
        raw_tables.append(rows)

        csv_file = os.path.join(csv_dir, f'table_page{page_number}_{i+1}.csv')
        with open(csv_file, 'w', newline='', encoding='utf-8') as f: csv.writer(f).writerows(rows)
        print(f"✅ Saved table CSV: {csv_file}")

        if len(rows) >= 2 and len(rows[0]) >= 2:
            dynamic_key = rows[0][0].strip() if rows[0][0].strip() else "ColumnHeader"
            column_labels = rows[0][1:]
            row_headers = [r[0] for r in rows[1:]]
            table_data = []
            for ci, label in enumerate(column_labels):
                entry = {dynamic_key: label}
                for ri, row in enumerate(rows[1:]):
                    if len(row) > ci + 1:
                        entry[row_headers[ri]] = row[ci + 1]
                table_data.append(entry)
            structured_tables.append(table_data)
        else:
            structured_tables.append(rows)

    return structured_tables, raw_tables, blocks, response

# === EXTRACT TEXT ===
def extract_text(blocks, image_path):
    lines = sorted([b for b in blocks if b['BlockType'] == 'LINE'], key=lambda x: x['Geometry']['BoundingBox']['Top'])
    image = cv2.imread(image_path)
    height, width = image.shape[:2]
    correct_coordinates(lines, width, height)

    line_links, complete, doc_paragraphs, doc_lines = {}, [], [], []
    for i in range(len(lines)):
        line_links[lines[i]['Id']] = None
        for j in range(i + 1, len(lines)):
            if analyze_height_for_para(lines[i], lines[j]) < 1 and analyze_width_for_para(lines[i], lines[j]) > 0.8:
                line_links[lines[i]['Id']] = lines[j]['Id']
                break

    for k in line_links:
        if k in complete: continue
        clist = [k]; complete.append(k)
        while line_links[clist[-1]]:
            nxt = line_links[clist[-1]]
            clist.append(nxt); complete.append(nxt)
        text = ' '.join(get_text_from_Id(lines, lid) for lid in clist)
        if len(clist) > 1 and is_long_sentence(text) and is_basic_svo(text):
            doc_paragraphs.append(clist)
        elif len(clist) == 1:
            doc_lines.append(clist[0])

    for group in doc_paragraphs + [[lid] for lid in doc_lines]:
        for lid in group:
            b = get_block_from_Id(lines, lid)
            if b:
                x, y, w_, h_ = b['Geometry']['BoundingBox'].values()
                image[y:y+h_, x:x+w_] = (255, 255, 255)

    doc_headings = list_headings(lines, 30)
    grouped_headings = [get_text_from_Id(lines, lid) for lid in doc_headings]
    grouped_paragraphs = [' '.join(get_text_from_Id(lines, lid) for lid in lst) for lst in doc_paragraphs]
    grouped_lines = [get_text_from_Id(lines, lid) for lid in doc_lines]
    return grouped_headings, grouped_paragraphs, grouped_lines, image

# === IMAGES ===
def extract_images(image_path, document_id, page_number):
    image = cv2.imread(image_path)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(gray, (5,5), 0)
    _, thresh = cv2.threshold(blurred, 200, 255, cv2.THRESH_BINARY_INV)
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    fig_dir = os.path.join(output_fig_root, document_id, "figures")
    os.makedirs(fig_dir, exist_ok=True)
    fig_paths = []
    for i, cnt in enumerate(contours):
        x, y, w, h = cv2.boundingRect(cnt)
        if w > 200 and h > 200:
            fig = image[y:y+h, x:x+w]
            fig_path = os.path.join(fig_dir, f"figure_page{page_number}_{i+1}.jpg")
            cv2.imwrite(fig_path, fig)
            fig_paths.append(fig_path)
    return fig_paths

# === DISPLAY COLLECTOR & FINAL DISPLAY ===
output_collector = []

def process_document(image_path, pdf_file, page_num, doc_id):
    structured_tables, raw_tables, blocks, raw_json = extract_tables(image_path, doc_id, page_num)
    headings, paras, lines, masked_img = extract_text(blocks, image_path)
    fig_paths = extract_images(image_path, doc_id, page_num)
    figs_bin = [Binary(open(fp, "rb").read()) for fp in fig_paths]

    # Store to MongoDB (with structured tables)
    collection = db[os.path.splitext(pdf_file)[0]]
    collection.insert_one({
        "document_id": doc_id,
        "page_number": page_num,
        "tables": structured_tables,
        "paragraphs": paras,
        "headings": headings,
        #"lines": lines,          #uncomment if u want to store lines in mongodb
        "figures": figs_bin,
        "raw_textract_json": json.loads(json.dumps(raw_json, default=str))
    })

    # Save labeled image
    out_dir = os.path.join(output_img_root, doc_id)
    os.makedirs(out_dir, exist_ok=True)
    labeled_path = os.path.join(out_dir, f"labeled_page_{page_num}.png")
    cv2.imwrite(labeled_path, masked_img)

    # Save to display collector
    output_collector.append({
        "pdf_file": pdf_file,
        "page_num": page_num,
        "headings": headings,
        "paragraphs": paras,
        #"lines": lines,       #uncomment if u want to display lines
        "tables": raw_tables  # ⬅️ Use raw extracted 2D rows here
    })

def display_final_output():
    for page_data in output_collector:
        display(Markdown(f"---\n## Extracted Content for {page_data['pdf_file']} (Page {page_data['page_num']})"))

        display(Markdown("**Headings:**"))
        if page_data["headings"]:
            for h in page_data["headings"]:
                display(Markdown(f"- {h}"))
        else:
            display(Markdown("_None_"))

        display(Markdown("**Paragraphs:**"))
        if page_data["paragraphs"]:
            for i, p in enumerate(page_data["paragraphs"], 1):
                display(Markdown(f"{i}. {p}"))
        else:
            display(Markdown("_None_"))

       # display(Markdown("**Lines:**"))
       # if page_data["lines"]:
        #    for i, l in enumerate(page_data["lines"], 1):
        #        display(Markdown(f"{i}. {l}"))
        #else:
         #   display(Markdown("_None_"))

        display(Markdown("**Tables:**"))
        if page_data["tables"]:
            for t in page_data["tables"]:
                display(pd.DataFrame(t) if isinstance(t, list) else Markdown("_Not a valid table_"))
        else:
            display(Markdown("_None_"))

def process_all_pdfs():
    for pdf in os.listdir(pdf_folder):
        if pdf.lower().endswith(".pdf"):
            path = os.path.join(pdf_folder, pdf)
            doc_id = os.path.splitext(pdf)[0]
            image_paths = convert_pdf_to_images(path, os.path.join(output_img_root, doc_id))
            for i, img_path in enumerate(image_paths):
                print(f"📄 Processing {img_path}...")
                corrected = correct_image_rotation(img_path)
                process_document(corrected, pdf, i + 1, doc_id)
    print("🎉 All PDFs processed successfully!")
    display_final_output()

if __name__ == "__main__":
    process_all_pdfs()

📄 Processing output_images\sample-2_sample-7_merged\sample-2_sample-7_merged_page_1.png...
✅ Saved table CSV: output_csv\sample-2_sample-7_merged\tables\table_page1_1.csv
✅ Saved table CSV: output_csv\sample-2_sample-7_merged\tables\table_page1_2.csv
📄 Processing output_images\sample-2_sample-7_merged\sample-2_sample-7_merged_page_2.png...
🔄 Rotated 90°: output_images\sample-2_sample-7_merged\sample-2_sample-7_merged_page_2.png
✅ Saved table CSV: output_csv\sample-2_sample-7_merged\tables\table_page2_1.csv
✅ Saved table CSV: output_csv\sample-2_sample-7_merged\tables\table_page2_2.csv
📄 Processing output_images\Sample2\Sample2_page_1.png...
📄 Processing output_images\Sample2\Sample2_page_2.png...
✅ Saved table CSV: output_csv\Sample2\tables\table_page2_1.csv
📄 Processing output_images\Sample2\Sample2_page_3.png...
🔄 Rotated 180°: output_images\Sample2\Sample2_page_3.png
✅ Saved table CSV: output_csv\Sample2\tables\table_page3_1.csv
✅ Saved table CSV: output_csv\Sample2\tables\table_pag

---
## Extracted Content for sample-2_sample-7_merged.pdf (Page 1)

**Headings:**

- YPP-1

- YOKOTA

...

**Paragraphs:**

_None_

**Tables:**

,0,1
0,HE42 F-1-980,UNIT LAVATORY
1,HE43 F-1-985,CATHODIC PROTECTION SYSTEM IN W.B.T.
2,HE44 -,LIFE JACKET INSTRUCTION


,0,1,2,3
0,,,CAPY TASOB,
1,,,,
2,,,,
